<a href="https://colab.research.google.com/github/agungyoga-tourism/desa-wisata-sustainable-tourism/blob/main/OLR_Thesis_Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set CRAN & instal paket yang diperlukan
options(repos = c(CRAN = "https://cloud.r-project.org"))
install.packages(c(
  "tidyverse",   # wrangling
  "MASS",        # polr (OLR)
  "brant",       # Brant test (uji proportional-odds untuk polr)
  "ordinal",     # clmm (mixed-effects ordinal)
  "VGAM",        # PPO (parallel=FALSE)
  "sandwich",    # (cluster-)robust vcov
  "lmtest",      # coeftest
  "broom",       # tabel rapi (tidy / glance)
  "clubSandwich",# cluster-robust var-cov alternatif
  "marginaleffects" # ringkas efek + robust vcov (opsional, membantu)
))

# Muat paket
library(tidyverse)
library(MASS)
library(brant)
library(ordinal)
library(VGAM)
library(sandwich)
library(lmtest)
library(broom)
library(clubSandwich)
library(marginaleffects)


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘ucminf’, ‘numDeriv’, ‘zoo’, ‘checkmate’, ‘Formula’, ‘insight’


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.5.2
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select



Attaching package: ‘ordinal’


The following object is masked from ‘package:dplyr’:

    slice


Loading required package: stats4

Loading required package: splines


Attaching package: ‘VGAM’


Th

In [2]:
# Ganti nama file sesuai yang Anda upload
df <- readr::read_csv("/content/SPSS Data per Oct 2025.csv")

# Cek cepat
glimpse(df, width = 120)


Rows: 571 Columns: 81
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Ethnicity
dbl (80): ID, Age_Raw, AgeGroup, Age_Group_New, Gender, Village, TVClassific...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 571
Columns: 81
$ ID                     <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, …
$ Age_Raw                <dbl> 17, 18, 18, 18, 16, 16, 18, 18, 18, 18, 18, 17, 18, 18, 18, 18, 18, 18, 17, 17, 17, 17,…
$ AgeGroup               <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ Age_Group_New          <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …
$ Gender                 <dbl> 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 3, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 3, …
$ Village                <dbl> 4, 9, 4, 4, 9, 8, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 5, 8, 8, 5, 8, 8, 8, 8, 8, 8, 8, 8, …
$ TVClassification       <dbl> 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, …
$ EduLevel_Recoded       <dbl> 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, …
$ Ethnicities_Bany

In [3]:
df %>% count(TV_Overall_Imp)

TV_Overall_Imp,n
<dbl>,<int>
1,192
2,364
3,13
4,1
5,1


In [4]:
library(dplyr)

# 1) Buat versi numerik yang dibalik (jangan menimpa variabel asli)
#    1->5, 2->4, 3->3, 4->2, 5->1
df <- df %>%
  mutate(overall_impact_pos_num = 6 - TV_Overall_Imp)

# 2) Cast ke faktor ordinal dengan label yang jelas
df <- df %>%
  mutate(overall_impact = factor(
    overall_impact_pos_num,
    levels = 1:5,
    ordered = TRUE,
    labels = c("Very negative","Negative","Neutral","Positive","Very positive")
  ))

# 3) Cek distribusi (harus kebalikan dari yang tadi)
df %>% count(overall_impact)


overall_impact,n
<ord>,<int>
Very negative,1
Negative,1
Neutral,13
Positive,364
Very positive,192


In [5]:
# Lihat kode yang ada, untuk memastikan mappingnya benar
df %>% count(Employ) %>% arrange(Employ)
df %>% count(Job_Tourism) %>% arrange(Job_Tourism)

# (opsional) cek jumlah desa/klaster
df %>% count(Village) %>% arrange(Village)


Employ,n
<dbl>,<int>
1,169
2,224
3,178


Job_Tourism,n
<dbl>,<int>
1,128
2,71
3,372


Village,n
<dbl>,<int>
1,42
2,71
3,36
4,36
5,95
6,80
7,38
8,98
9,75


In [8]:
library(dplyr)
library(MASS)

# 1) subset variabel dengan dplyr::select (bukan MASS::select)
dat_min <- df2 %>%
  dplyr::select(overall_impact, emp_status, jobrel) %>%
  tidyr::drop_na()

nrow(dat_min)

# 2) fit OLR minimal
m_min <- MASS::polr(overall_impact ~ emp_status + jobrel,
                    data = dat_min, Hess = TRUE, method = "logistic")

# 3) OR, CI 95%, p-value (Wald)
ct <- coef(summary(m_min))                # Value, Std. Error, t value
pvals <- 2 * pnorm(abs(ct[, "t value"]), lower.tail = FALSE)

res_all <- data.frame(
  Term  = rownames(ct),
  OR    = exp(ct[, "Value"]),
  L95   = exp(ct[, "Value"] - 1.96 * ct[, "Std. Error"]),
  U95   = exp(ct[, "Value"] + 1.96 * ct[, "Std. Error"]),
  p     = pvals,
  row.names = NULL
)

res_effects <- res_all %>%
  dplyr::filter(grepl("^(emp_status|jobrel)", Term)) %>%
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"        ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"      ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"   ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"     ~ "Directly vs Unrelated",
      TRUE ~ Term
    )
  ) %>%
  dplyr::select(Comparison, OR, L95, U95, p)

print(res_effects, digits = 4)


[1] 571

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


                Comparison     OR    L95   U95         p
1   Part-time vs Full-time 0.6671 0.4403 1.011 5.617e-02
2 Not working vs Full-time 0.6793 0.4304 1.072 9.673e-02
3  Indirectly vs Unrelated 2.4840 1.4537 4.244 8.729e-04
4    Directly vs Unrelated 2.6760 1.7446 4.105 6.493e-06


In [9]:
# Pastikan dat_min menyertakan kolom Weight
dat_min_w <- df2 %>%
  dplyr::select(overall_impact, emp_status, jobrel, Weight) %>%
  tidyr::drop_na()

# Fit OLR (weighted) – MASS::polr menerima case weights
m_min_w <- MASS::polr(overall_impact ~ emp_status + jobrel,
                      data = dat_min_w, weights = Weight,
                      Hess = TRUE, method = "logistic")

# Ekstrak OR, CI 95% (Wald), dan p-value
ct_w   <- coef(summary(m_min_w))
p_w    <- 2 * pnorm(abs(ct_w[, "t value"]), lower.tail = FALSE)

res_all_w <- data.frame(
  Term = rownames(ct_w),
  OR   = exp(ct_w[, "Value"]),
  L95  = exp(ct_w[, "Value"] - 1.96 * ct_w[, "Std. Error"]),
  U95  = exp(ct_w[, "Value"] + 1.96 * ct_w[, "Std. Error"]),
  p    = p_w,
  row.names = NULL
)

res_effects_w <- res_all_w %>%
  dplyr::filter(grepl("^(emp_status|jobrel)", Term)) %>%
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"        ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"      ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"   ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"     ~ "Directly vs Unrelated",
      TRUE ~ Term
    )
  ) %>%
  dplyr::select(Comparison, OR, L95, U95, p)

print(res_effects_w, digits = 4)


Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


                Comparison     OR    L95    U95         p
1   Part-time vs Full-time 0.5517 0.3696 0.8234 3.602e-03
2 Not working vs Full-time 0.6200 0.3863 0.9951 4.767e-02
3  Indirectly vs Unrelated 3.4528 2.0735 5.7496 1.909e-06
4    Directly vs Unrelated 2.6748 1.7768 4.0266 2.427e-06


In [10]:
library(brant)
# model tanpa weight tapi dengan spesifikasi identik
m_min_unw <- MASS::polr(overall_impact ~ emp_status + jobrel,
                        data = dat_min, Hess = TRUE, method = "logistic")

# Brant (test of parallel lines / proportional odds)
br <- brant(m_min_unw)
br


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


------------------------------------------------------------ 
Test for			X2	df	probability 
------------------------------------------------------------ 
Omnibus				12.01	12	0.45
emp_statusPart-time		0.25	3	0.97
emp_statusNot working		1.93	3	0.59
jobrelIndirectly related	9.05	3	0.03
jobrelDirectly related	3.7	3	0.3
------------------------------------------------------------ 

H0: Parallel Regression Assumption holds


Warning message in brant(m_min_unw):
“19 combinations in table(dv,ivs) do not occur. Because of that, the test results might be invalid.”


,X2,df,probability
Omnibus,12.007439,12,0.44508235
emp_statusPart-time,0.252128,3,0.96876522
emp_statusNot working,1.933682,3,0.58628471
jobrelIndirectly related,9.053037,3,0.02859398
jobrelDirectly related,3.704614,3,0.29517778


In [11]:
library(dplyr)
library(MASS)

# 1) Buat faktor tv_class sesuai mapping yang Anda berikan
df3 <- df2 %>%
  mutate(
    tv_class_chr = case_when(
      TVClassification == 4 ~ "Advanced",        # code 4 = Advance
      TVClassification == 3 ~ "Developing",
      TVClassification == 2 ~ "Piloting",
      TVClassification == 1 ~ "Not classified",
      TRUE ~ NA_character_
    ),
    tv_class = factor(tv_class_chr,
                      levels = c("Advanced","Piloting","Developing","Not classified")) # referensi = Advanced
  )

# Cek distribusi level
df3 %>% count(tv_class)

# 2) Dataset untuk model (pakai weights)
dat_m2 <- df3 %>%
  dplyr::select(overall_impact, emp_status, jobrel, tv_class, Weight) %>%
  tidyr::drop_na()

nrow(dat_m2)

# 3) Fit OLR berbobot
m2_w <- MASS::polr(overall_impact ~ emp_status + jobrel + tv_class,
                   data = dat_m2, weights = Weight,
                   Hess = TRUE, method = "logistic")

# 4) Ekstrak OR, CI 95%, p-value (Wald)
ct2 <- coef(summary(m2_w))
p2  <- 2 * pnorm(abs(ct2[, "t value"]), lower.tail = FALSE)

res_all2 <- data.frame(
  Term = rownames(ct2),
  OR   = exp(ct2[, "Value"]),
  L95  = exp(ct2[, "Value"] - 1.96 * ct2[, "Std. Error"]),
  U95  = exp(ct2[, "Value"] + 1.96 * ct2[, "Std. Error"]),
  p    = p2,
  row.names = NULL
)

res_effects2 <- res_all2 %>%
  dplyr::filter(grepl("^(emp_status|jobrel|tv_class)", Term)) %>%
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"          ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"        ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"     ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"       ~ "Directly vs Unrelated",
      Term == "tv_classPiloting"             ~ "Piloting vs Advanced",
      Term == "tv_classDeveloping"           ~ "Developing vs Advanced",
      Term == "tv_classNot classified"       ~ "Not classified vs Advanced",
      TRUE ~ Term
    )
  ) %>%
  dplyr::select(Comparison, OR, L95, U95, p)

print(res_effects2, digits = 4)


tv_class,n
<fct>,<int>
Advanced,113
Piloting,267
Developing,80
Not classified,111


[1] 571

Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


                  Comparison     OR    L95    U95         p
1     Part-time vs Full-time 0.5335 0.3555 0.8006 2.413e-03
2   Not working vs Full-time 0.5348 0.3285 0.8707 1.185e-02
3    Indirectly vs Unrelated 3.4503 2.0437 5.8250 3.568e-06
4      Directly vs Unrelated 2.4962 1.6439 3.7904 1.766e-05
5       Piloting vs Advanced 0.5718 0.3391 0.9640 3.594e-02
6     Developing vs Advanced 0.3620 0.1814 0.7223 3.942e-03
7 Not classified vs Advanced 0.8659 0.4601 1.6296 6.554e-01


In [12]:
library(MASS)
library(brant)

# dataset tanpa missing utk 3 prediktor
dat_m2_unw <- df3 %>%
  dplyr::select(overall_impact, emp_status, jobrel, tv_class) %>%
  tidyr::drop_na()

m2_unw <- MASS::polr(overall_impact ~ emp_status + jobrel + tv_class,
                     data = dat_m2_unw, Hess = TRUE, method = "logistic")

br2 <- brant(m2_unw)
br2


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


------------------------------------------------------------ 
Test for			X2	df	probability 
------------------------------------------------------------ 
Omnibus				18.4	21	0.62
emp_statusPart-time		0.21	3	0.98
emp_statusNot working		2.02	3	0.57
jobrelIndirectly related	8.43	3	0.04
jobrelDirectly related	3.15	3	0.37
tv_classPiloting		0.03	3	1
tv_classDeveloping		0.68	3	0.88
tv_classNot classified	1.43	3	0.7
------------------------------------------------------------ 

H0: Parallel Regression Assumption holds


Warning message in brant(m2_unw):
“108 combinations in table(dv,ivs) do not occur. Because of that, the test results might be invalid.”


,X2,df,probability
Omnibus,18.39744515,21,0.62373271
emp_statusPart-time,0.21381705,3,0.97532852
emp_statusNot working,2.01639200,3,0.56901146
jobrelIndirectly related,8.42967049,3,0.03791822
jobrelDirectly related,3.15035211,3,0.36901971
tv_classPiloting,0.02716707,3,0.99881874
tv_classDeveloping,0.68420402,3,0.87691288
tv_classNot classified,1.42575843,3,0.69950808


In [13]:
# Cek rentang umur dan distribusinya
df %>% summarise(N = n(), min_age = min(Age_Raw, na.rm=TRUE),
                 p25 = quantile(Age_Raw, .25, na.rm=TRUE),
                 median = median(Age_Raw, na.rm=TRUE),
                 p75 = quantile(Age_Raw, .75, na.rm=TRUE),
                 max_age = max(Age_Raw, na.rm=TRUE))

# Lihat kode pendidikan
df %>% count(EduLevel_Recoded) %>% arrange(EduLevel_Recoded)


N,min_age,p25,median,p75,max_age
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
571,16,23,32,42,60


EduLevel_Recoded,n
<dbl>,<int>
1,45
2,109
3,277
4,139
NA,1


In [16]:
# Pakai dataset terakhir yang kita pakai untuk model (dat_m3)
# Jika belum ada di sesi Anda, buat ulang:
dat_m3 <- df4 %>%
  dplyr::select(overall_impact, emp_status, jobrel, tv_class, Age_Raw, education, Weight) %>%
  tidyr::drop_na()

# 1) Cek bobot (Weight): NA, nol, distribusi
summary(dat_m3$Weight)
c(na_count = sum(is.na(dat_m3$Weight)),
  zero_count = sum(dat_m3$Weight == 0),
  min_w = min(dat_m3$Weight, na.rm=TRUE),
  max_w = max(dat_m3$Weight, na.rm=TRUE))
quantile(dat_m3$Weight, c(0, .01, .5, .99, 1), na.rm=TRUE)

# 2) Cek sel nol yang bisa bikin separasi
with(dat_m3, addmargins(table(overall_impact, emp_status)))
with(dat_m3, addmargins(table(overall_impact, jobrel)))
with(dat_m3, addmargins(table(overall_impact, tv_class)))

# 3) Cek korelasi/persilangan cepat: outcome paling ekstrem
subset_extreme <- dat_m3 %>% filter(overall_impact %in% c("Very negative","Negative"))
nrow(subset_extreme)
subset_extreme %>% count(emp_status)
subset_extreme %>% count(jobrel)
subset_extreme %>% count(tv_class)


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.531   0.574   0.990   1.021   1.005   2.398 

na_count zero_count      min_w      max_w 
     0.000      0.000      0.531      2.398

0%    1%   50%   99%  100% 
0.531 0.531 0.990 2.398 2.398

,Full-time,Part-time,Not working,Sum
Very negative,0,1,0,1
Negative,0,0,1,1
Neutral,3,3,7,13
Positive,105,146,113,364
Very positive,69,74,48,191
Sum,177,224,169,570


,Unrelated,Indirectly related,Directly related,Sum
Very negative,0,1,0,1
Negative,1,0,0,1
Neutral,8,2,3,13
Positive,268,33,63,364
Very positive,94,35,62,191
Sum,371,71,128,570


,Advanced,Piloting,Developing,Not classified,Sum
Very negative,0,1,0,0,1
Negative,0,0,0,1,1
Neutral,3,8,2,0,13
Positive,59,171,62,72,364
Very positive,51,86,16,38,191
Sum,113,266,80,111,570


[1] 2

emp_status,n
<fct>,<int>
Part-time,1
Not working,1


jobrel,n
<fct>,<int>
Unrelated,1
Indirectly related,1


tv_class,n
<fct>,<int>
Piloting,1
Not classified,1


In [17]:
library(ordinal)
library(dplyr)

# Pastikan dat_m3 ada (outcome + prediktor + Weight, tanpa NA)
dat_m3 <- df4 %>%
  dplyr::select(overall_impact, emp_status, jobrel, tv_class, Age_Raw, education, Weight) %>%
  tidyr::drop_na()

# 1) Fit cumulative link model (logit) dengan weights
m3_clm <- clm(
  overall_impact ~ emp_status + jobrel + tv_class + Age_Raw + education,
  data = dat_m3,
  weights = Weight,
  link = "logit"
)

# 2) Ambil koefisien lokasi (bukan cut-points), lalu hitung OR, CI Wald 95%, p (z-test)
co <- summary(m3_clm)$coefficients   # termasuk threshold & lokasi
# Ambil hanya baris efek prediktor (bukan threshold "Very negative|Negative", dst.)
keep <- grepl("^(emp_status|jobrel|tv_class|Age_Raw|education)", rownames(co))

co_eff <- co[keep, , drop = FALSE]
beta   <- co_eff[, "Estimate"]
se     <- co_eff[, "Std. Error"]
zval   <- beta / se
pval   <- 2 * pnorm(abs(zval), lower.tail = FALSE)

res_effects3_clm <- data.frame(
  Term  = rownames(co_eff),
  OR    = exp(beta),
  L95   = exp(beta - 1.96 * se),
  U95   = exp(beta + 1.96 * se),
  p     = pval,
  row.names = NULL
) |>
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"          ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"        ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"     ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"       ~ "Directly vs Unrelated",
      Term == "tv_classPiloting"             ~ "Piloting vs Advanced",
      Term == "tv_classDeveloping"           ~ "Developing vs Advanced",
      Term == "tv_classNot classified"       ~ "Not classified vs Advanced",
      Term == "Age_Raw"                      ~ "Age (per year)",
      Term == "educationUpperSec"            ~ "UpperSec vs Higher",
      Term == "educationLowerSec"            ~ "LowerSec vs Higher",
      Term == "educationNoPrimary"           ~ "No/Primary vs Higher",
      TRUE ~ Term
    )
  ) |>
  dplyr::select(Comparison, OR, L95, U95, p)

print(res_effects3_clm, digits = 4)


                   Comparison     OR     L95    U95         p
1      Part-time vs Full-time 0.6188 0.39886 0.9599 3.214e-02
2    Not working vs Full-time 0.7118 0.41292 1.2269 2.210e-01
3     Indirectly vs Unrelated 3.2030 1.88906 5.4308 1.552e-05
4       Directly vs Unrelated 2.5110 1.64377 3.8357 2.052e-05
5        Piloting vs Advanced 0.5896 0.34418 1.0100 5.441e-02
6      Developing vs Advanced 0.3195 0.15838 0.6446 1.440e-03
7  Not classified vs Advanced 0.8697 0.45496 1.6624 6.727e-01
8              Age (per year) 1.0179 0.99955 1.0367 5.598e-02
9          UpperSec vs Higher 0.9519 0.59958 1.5111 8.342e-01
10         LowerSec vs Higher 0.6904 0.38301 1.2444 2.177e-01
11       No/Primary vs Higher 0.2246 0.09243 0.5459 9.804e-04


In [20]:
library(ordinal)
library(dplyr)

# Dataset tanpa weight untuk uji PO
dat_m3_unw <- df4 %>%
  dplyr::select(overall_impact, emp_status, jobrel, tv_class, Age_Raw, education) %>%
  tidyr::drop_na()

# Fit model clm (tanpa weights), spesifikasi identik
m3_clm_unw <- clm(
  overall_impact ~ emp_status + jobrel + tv_class + Age_Raw + education,
  data = dat_m3_unw,
  link = "logit"
)

# Uji proportional odds ala ordinal
nominal_test(m3_clm_unw)


,Df,logLik,AIC,LRT,Pr(>Chi)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
<none>,NA,-410.4438,850.8875,NA,NA
emp_status,NA,NA,NA,NA,NA
jobrel,NA,NA,NA,NA,NA
tv_class,NA,NA,NA,NA,NA
Age_Raw,NA,NA,NA,NA,NA
education,NA,NA,NA,NA,NA


In [23]:
library(dplyr)

# Rehitung tabel efek dari m4_clm lalu pakai dplyr::select
co <- summary(m4_clm)$coefficients
keep <- grepl("^(emp_status|jobrel|tv_class|Age_Raw|education|industry)", rownames(co))
co_eff <- co[keep, , drop = FALSE]

beta <- co_eff[, "Estimate"]; se <- co_eff[, "Std. Error"]
zval <- beta / se; pval <- 2 * pnorm(abs(zval), lower.tail = FALSE)

res_effects4_clm <- data.frame(
  Term = rownames(co_eff),
  OR   = exp(beta),
  L95  = exp(beta - 1.96*se),
  U95  = exp(beta + 1.96*se),
  p    = pval,
  row.names = NULL
) %>%
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"            ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"          ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"       ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"         ~ "Directly vs Unrelated",
      Term == "tv_classPiloting"               ~ "Piloting vs Advanced",
      Term == "tv_classDeveloping"             ~ "Developing vs Advanced",
      Term == "tv_classNot classified"         ~ "Not classified vs Advanced",
      Term == "Age_Raw"                        ~ "Age (per year)",
      Term == "educationUpperSec"              ~ "UpperSec vs Higher",
      Term == "educationLowerSec"              ~ "LowerSec vs Higher",
      Term == "educationNoPrimary"             ~ "No/Primary vs Higher",
      Term == "industryAgriculture/fisheries"  ~ "Agriculture/fisheries vs Other",
      Term == "industryManufacturing"          ~ "Manufacturing vs Other",
      Term == "industryPublic sector"          ~ "Public sector vs Other",
      Term == "industryServices"               ~ "Services vs Other",
      Term == "industryTourism"                ~ "Tourism vs Other",
      TRUE ~ Term
    )
  ) %>%
  dplyr::select(Comparison, OR, L95, U95, p)

print(res_effects4_clm, digits = 4)


                       Comparison     OR     L95    U95         p
1          Part-time vs Full-time 0.6169 0.39259 0.9693 3.615e-02
2        Not working vs Full-time 0.6786 0.39010 1.1804 1.698e-01
3         Indirectly vs Unrelated 3.1695 1.80810 5.5560 5.621e-05
4           Directly vs Unrelated 2.4168 1.45261 4.0210 6.802e-04
5            Piloting vs Advanced 0.5966 0.34640 1.0277 6.267e-02
6          Developing vs Advanced 0.3156 0.15603 0.6385 1.336e-03
7      Not classified vs Advanced 0.8669 0.45165 1.6637 6.675e-01
8                  Age (per year) 1.0160 0.99711 1.0354 9.728e-02
9              UpperSec vs Higher 0.9798 0.61179 1.5693 9.325e-01
10             LowerSec vs Higher 0.6919 0.37897 1.2632 2.305e-01
11           No/Primary vs Higher 0.2237 0.09064 0.5521 1.159e-03
12 Agriculture/fisheries vs Other 0.9721 0.58710 1.6095 9.123e-01
13         Manufacturing vs Other 0.3262 0.10215 1.0414 5.855e-02
14         Public sector vs Other 0.9767 0.36665 2.6016 9.623e-01
15        

In [24]:
library(dplyr)
library(ordinal)

# 1) Tambah faktor income, residence, gender (dengan referensi yang diminta)
df6 <- df5 %>%
  mutate(
    income_chr = case_when(
      Income == 6 ~ "Above Rp 8,442,602",
      Income == 5 ~ "Rp 2,638,628–Rp 8,442,602",
      Income == 4 ~ "Rp 1,688,520–Rp 2,638,628",
      Income == 3 ~ "Rp 448,928–Rp 1,688,520",
      Income == 2 ~ "Rp 351,957–Rp 448,928",
      Income == 1 ~ "Below Rp 351,957",
      TRUE ~ NA_character_
    ),
    income = factor(
      income_chr,
      levels = c("Above Rp 8,442,602",
                 "Rp 2,638,628–Rp 8,442,602",
                 "Rp 1,688,520–Rp 2,638,628",
                 "Rp 448,928–Rp 1,688,520",
                 "Rp 351,957–Rp 448,928",
                 "Below Rp 351,957")
    ),
    residence_chr = case_when(
      Resid_Time == 5 ~ "≥ 20 years",
      Resid_Time == 4 ~ "15–19 years",
      Resid_Time == 3 ~ "10–14 years",
      Resid_Time == 2 ~ "5–9 years",
      Resid_Time == 1 ~ "Under 5 years",
      TRUE ~ NA_character_
    ),
    residence_len = factor(
      residence_chr,
      levels = c("≥ 20 years","15–19 years","10–14 years","5–9 years","Under 5 years")
    ),
    gender_chr = case_when(
      Gender == 3 ~ "Other",
      Gender == 1 ~ "Male",
      Gender == 2 ~ "Female",
      TRUE ~ NA_character_
    ),
    gender = factor(gender_chr, levels = c("Other","Male","Female"))
  )

# Cek ringkas
df6 %>% count(income)
df6 %>% count(residence_len)
df6 %>% count(gender)

# 2) Siapkan data untuk model penuh (pakai weights)
dat_full <- df6 %>%
  dplyr::select(
    overall_impact, emp_status, jobrel, tv_class,
    Age_Raw, education, industry,
    income, residence_len, gender,
    Weight
  ) %>%
  tidyr::drop_na()

nrow(dat_full)

# 3) Fit CLM (logit) berbobot – spesifikasi mendekati Tabel 4.6
m_full <- clm(
  overall_impact ~ emp_status + jobrel + tv_class +
    Age_Raw + education + industry +
    income + residence_len + gender,
  data = dat_full, weights = Weight, link = "logit"
)

# 4) Ekstrak OR, CI95, p untuk semua prediktor (bukan threshold)
co <- summary(m_full)$coefficients
keep <- grepl("^(emp_status|jobrel|tv_class|Age_Raw|education|industry|income|residence_len|gender)", rownames(co))
co_eff <- co[keep, , drop = FALSE]

beta <- co_eff[, "Estimate"]; se <- co_eff[, "Std. Error"]
zval <- beta / se; pval <- 2 * pnorm(abs(zval), lower.tail = FALSE)

res_full <- data.frame(
  Term = rownames(co_eff),
  OR   = exp(beta),
  L95  = exp(beta - 1.96*se),
  U95  = exp(beta + 1.96*se),
  p    = pval,
  row.names = NULL
) %>%
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"              ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"            ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"         ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"           ~ "Directly vs Unrelated",
      Term == "tv_classPiloting"                 ~ "Piloting vs Advanced",
      Term == "tv_classDeveloping"               ~ "Developing vs Advanced",
      Term == "tv_classNot classified"           ~ "Not classified vs Advanced",
      Term == "Age_Raw"                          ~ "Age (per year)",
      Term == "educationUpperSec"                ~ "UpperSec vs Higher",
      Term == "educationLowerSec"                ~ "LowerSec vs Higher",
      Term == "educationNoPrimary"               ~ "No/Primary vs Higher",
      Term == "industryAgriculture/fisheries"    ~ "Agriculture/fisheries vs Other",
      Term == "industryManufacturing"            ~ "Manufacturing vs Other",
      Term == "industryPublic sector"            ~ "Public sector vs Other",
      Term == "industryServices"                 ~ "Services vs Other",
      Term == "industryTourism"                  ~ "Tourism vs Other",
      grepl("^income", Term)                     ~ sub("^income", "", Term),
      grepl("^residence_len", Term)              ~ sub("^residence_len", "", Term),
      grepl("^gender", Term)                     ~ sub("^gender", "", Term),
      TRUE ~ Term
    )
  ) %>%
  dplyr::select(Comparison, OR, L95, U95, p) %>%
  dplyr::arrange(match(Comparison, c(
    "Part-time vs Full-time","Not working vs Full-time",
    "Indirectly vs Unrelated","Directly vs Unrelated",
    "Piloting vs Advanced","Developing vs Advanced","Not classified vs Advanced",
    "Age (per year)",
    "UpperSec vs Higher","LowerSec vs Higher","No/Primary vs Higher",
    "Agriculture/fisheries vs Other","Manufacturing vs Other","Public sector vs Other","Services vs Other","Tourism vs Other",
    " Rp 2,638,628–Rp 8,442,602"," Rp 1,688,520–Rp 2,638,628"," Rp 448,928–Rp 1,688,520"," Rp 351,957–Rp 448,928"," Below Rp 351,957",
    " 15–19 years"," 10–14 years"," 5–9 years"," Under 5 years",
    " Male"," Female"
  )))

print(res_full, digits = 4)


income,n
<fct>,<int>
"Above Rp 8,442,602",9
"Rp 2,638,628–Rp 8,442,602",48
"Rp 1,688,520–Rp 2,638,628",114
"Rp 448,928–Rp 1,688,520",189
"Rp 351,957–Rp 448,928",82
"Below Rp 351,957",129


residence_len,n
<fct>,<int>
≥ 20 years,369
15–19 years,101
10–14 years,44
5–9 years,33
Under 5 years,24


gender,n
<fct>,<int>
Other,5
Male,265
Female,301


[1] 570

                       Comparison     OR     L95    U95         p
1          Part-time vs Full-time 0.4997 0.30312 0.8237 0.0065146
2        Not working vs Full-time 0.4316 0.22987 0.8103 0.0089348
3         Indirectly vs Unrelated 2.1124 1.14958 3.8817 0.0159941
4           Directly vs Unrelated 2.5734 1.51996 4.3568 0.0004339
5            Piloting vs Advanced 0.5130 0.28763 0.9150 0.0237677
6          Developing vs Advanced 0.2805 0.13473 0.5838 0.0006767
7      Not classified vs Advanced 0.8794 0.44833 1.7249 0.7084295
8                  Age (per year) 1.0157 0.99531 1.0364 0.1323131
9              UpperSec vs Higher 0.8706 0.53069 1.4281 0.5831251
10             LowerSec vs Higher 0.6350 0.33647 1.1983 0.1610166
11           No/Primary vs Higher 0.2079 0.08291 0.5211 0.0008074
12 Agriculture/fisheries vs Other 1.0131 0.59621 1.7215 0.9616501
13         Manufacturing vs Other 0.4128 0.12793 1.3319 0.1387533
14         Public sector vs Other 0.8739 0.30872 2.4739 0.7996147
15        

In [28]:
library(dplyr)
library(sandwich)
library(lmtest)

# === 1) Buat ulang data yang dipakai model + kolom cluster (Village) ===
dat_full_cl <- df6 %>%
  dplyr::select(
    overall_impact, emp_status, jobrel, tv_class,
    Age_Raw, education, industry,
    income, residence_len, gender,
    Weight, Village
  ) %>%
  tidyr::drop_na()

# Cek baris harus sama dengan model (m_full)
cat("N dat_full_cl =", nrow(dat_full_cl), "\n")
cat("N m_full      =", nrow(model.frame(m_full)), "\n")

# === 2) Cluster-robust vcov by Village dari model clm berbobot (m_full) ===
V_cl <- sandwich::vcovCL(m_full, cluster = dat_full_cl$Village)

ct <- lmtest::coeftest(m_full, vcov = V_cl)

# Ambil hanya efek prediktor (buang cutpoints/threshold)
keep <- grepl("^(emp_status|jobrel|tv_class|Age_Raw|education|industry|income|residence_len|gender)", rownames(ct))
tab  <- ct[keep, , drop = FALSE]

beta <- tab[, 1]; se <- tab[, 2]; p <- tab[, 4]
OR   <- exp(beta); L95 <- exp(beta - 1.96*se); U95 <- exp(beta + 1.96*se)

res_cl_vil <- data.frame(
  Term = rownames(tab),
  OR, L95, U95, p,
  row.names = NULL
) %>%
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"              ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"            ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"         ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"           ~ "Directly vs Unrelated",
      Term == "tv_classPiloting"                 ~ "Piloting vs Advanced",
      Term == "tv_classDeveloping"               ~ "Developing vs Advanced",
      Term == "tv_classNot classified"           ~ "Not classified vs Advanced",
      Term == "Age_Raw"                          ~ "Age (per year)",
      Term == "educationUpperSec"                ~ "UpperSec vs Higher",
      Term == "educationLowerSec"                ~ "LowerSec vs Higher",
      Term == "educationNoPrimary"               ~ "No/Primary vs Higher",
      Term == "industryAgriculture/fisheries"    ~ "Agriculture/fisheries vs Other",
      Term == "industryManufacturing"            ~ "Manufacturing vs Other",
      Term == "industryPublic sector"            ~ "Public sector vs Other",
      Term == "industryServices"                 ~ "Services vs Other",
      Term == "industryTourism"                  ~ "Tourism vs Other",
      grepl("^income", Term)                     ~ sub("^income", "", Term),
      grepl("^residence_len", Term)              ~ sub("^residence_len", "", Term),
      grepl("^gender", Term)                     ~ sub("^gender", "", Term),
      TRUE ~ Term
    )
  ) %>%
  dplyr::select(Comparison, OR, L95, U95, p)

# Tampilkan 4 efek utama dulu
print(
  res_cl_vil %>% dplyr::filter(Comparison %in% c(
    "Part-time vs Full-time","Not working vs Full-time",
    "Indirectly vs Unrelated","Directly vs Unrelated"
  )),
  digits = 4
)

# (opsional) Semua efek:
# print(res_cl_vil, digits = 4)


N dat_full_cl = 570 
N m_full      = 570 
                Comparison     OR    L95    U95         p
1   Part-time vs Full-time 0.4997 0.2470 1.0107 0.0540712
2 Not working vs Full-time 0.4316 0.2199 0.8472 0.0149168
3  Indirectly vs Unrelated 2.1124 1.3445 3.3190 0.0012502
4    Directly vs Unrelated 2.5734 1.4775 4.4820 0.0008978


In [29]:
library(ordinal)
library(dplyr)

# Data untuk clmm (tanpa weight), lengkap dgn Village
dat_clmm <- df6 %>%
  dplyr::select(
    overall_impact, emp_status, jobrel, tv_class,
    Age_Raw, education, industry,
    income, residence_len, gender,
    Village
  ) %>%
  tidyr::drop_na()

# Fit mixed-effects ordinal logit (random intercept by Village)
m_clmm <- clmm(
  overall_impact ~ emp_status + jobrel + tv_class +
    Age_Raw + education + industry +
    income + residence_len + gender +
    (1 | Village),
  data = dat_clmm,
  link = "logit",
  nAGQ = 10,               # kuadratur adaptif (stabil)
  Hess = TRUE,
  control = clmm.control(maxIter = 1000, gradTol = 1e-6)
)

summary(m_clmm)

# Ekstrak OR, CI Wald 95%, p untuk efek tetap (bukan threshold)
co <- summary(m_clmm)$coefficients  # kolom: Estimate, Std. Error, z value
keep <- grepl("^(emp_status|jobrel|tv_class|Age_Raw|education|industry|income|residence_len|gender)", rownames(co))
co_eff <- co[keep, , drop = FALSE]

beta <- co_eff[, "Estimate"]
se   <- co_eff[, "Std. Error"]
z    <- beta / se
p    <- 2 * pnorm(abs(z), lower.tail = FALSE)

res_clmm <- data.frame(
  Term = rownames(co_eff),
  OR   = exp(beta),
  L95  = exp(beta - 1.96 * se),
  U95  = exp(beta + 1.96 * se),
  p    = p,
  row.names = NULL
) %>%
  dplyr::mutate(
    Comparison = dplyr::case_when(
      Term == "emp_statusPart-time"              ~ "Part-time vs Full-time",
      Term == "emp_statusNot working"            ~ "Not working vs Full-time",
      Term == "jobrelIndirectly related"         ~ "Indirectly vs Unrelated",
      Term == "jobrelDirectly related"           ~ "Directly vs Unrelated",
      Term == "tv_classPiloting"                 ~ "Piloting vs Advanced",
      Term == "tv_classDeveloping"               ~ "Developing vs Advanced",
      Term == "tv_classNot classified"           ~ "Not classified vs Advanced",
      Term == "Age_Raw"                          ~ "Age (per year)",
      Term == "educationUpperSec"                ~ "UpperSec vs Higher",
      Term == "educationLowerSec"                ~ "LowerSec vs Higher",
      Term == "educationNoPrimary"               ~ "No/Primary vs Higher",
      Term == "industryAgriculture/fisheries"    ~ "Agriculture/fisheries vs Other",
      Term == "industryManufacturing"            ~ "Manufacturing vs Other",
      Term == "industryPublic sector"            ~ "Public sector vs Other",
      Term == "industryServices"                 ~ "Services vs Other",
      Term == "industryTourism"                  ~ "Tourism vs Other",
      grepl("^income", Term)                     ~ sub("^income", "", Term),
      grepl("^residence_len", Term)              ~ sub("^residence_len", "", Term),
      grepl("^gender", Term)                     ~ sub("^gender", "", Term),
      TRUE ~ Term
    )
  ) %>%
  dplyr::select(Comparison, OR, L95, U95, p)

# Tampilkan 4 efek utama
print(res_clmm %>% dplyr::filter(Comparison %in% c(
  "Part-time vs Full-time","Not working vs Full-time",
  "Indirectly vs Unrelated","Directly vs Unrelated"
)), digits = 4)

# (opsional) seluruh tabel:
# print(res_clmm, digits = 4)


Cumulative Link Mixed Model fitted with the adaptive Gauss-Hermite 
quadrature approximation with 10 quadrature points 

formula: 
overall_impact ~ emp_status + jobrel + tv_class + Age_Raw + education +  
    industry + income + residence_len + gender + (1 | Village)
data:    dat_clmm

 link  threshold nobs logLik  AIC    niter       max.grad cond.H 
 logit flexible  570  -397.85 859.70 4440(13329) 1.19e-03 8.4e+05

Random effects:
 Groups  Name        Variance Std.Dev.
 Village (Intercept) 0.1135   0.3368  
Number of groups:  Village 9 

Coefficients:
                                Estimate Std. Error z value Pr(>|z|)   
emp_statusPart-time             -0.42260    0.26286  -1.608  0.10791   
emp_statusNot working           -0.63456    0.31172  -2.036  0.04178 * 
jobrelIndirectly related         0.68807    0.31461   2.187  0.02874 * 
jobrelDirectly related           0.84604    0.28269   2.993  0.00276 **
tv_classPiloting                -0.59957    0.39928  -1.502  0.13319   
tv_classD

                Comparison     OR    L95    U95        p
1   Part-time vs Full-time 0.6553 0.3915 1.0970 0.107907
2 Not working vs Full-time 0.5302 0.2878 0.9767 0.041782
3  Indirectly vs Unrelated 1.9899 1.0740 3.6866 0.028739
4    Directly vs Unrelated 2.3304 1.3391 4.0556 0.002764


In [30]:
library(ordinal)
library(sandwich)
library(lmtest)
library(dplyr)

# Pastikan dat_full (dari langkah sebelumnya) dan m_full (PO weighted) sudah ada

# 1) Fit PPO: relaksasi PO khusus untuk jobrel
m_ppo <- clm(
  overall_impact ~ emp_status + jobrel + tv_class +
    Age_Raw + education + industry +
    income + residence_len + gender,
  nominal = ~ jobrel,               # <-- PPO utk jobrel saja
  data = dat_full,
  weights = Weight,
  link = "logit"
)

# 2) Perbandingan PO vs PPO (uji LRT; informasi model)
anova(m_full, m_ppo)  # jika signifikan → PPO memberi peningkatan kecocokan

# 3) Cluster-robust SE by Village untuk m_ppo
Vcl_ppo <- sandwich::vcovCL(m_ppo, cluster = dat_full$Village)
ct_ppo  <- lmtest::coeftest(m_ppo, vcov = Vcl_ppo)

# 4) Ambil koefisien PREDIKTOR SAJA (buang threshold)
keep_all <- grepl("^(emp_status|jobrel|tv_class|Age_Raw|education|industry|income|residence_len|gender)",
                  rownames(ct_ppo))
tab_all  <- ct_ppo[keep_all, , drop = FALSE]

# 4a) Ringkasan efek utama lain (emp_status dsb.) → tetap satu OR (seperti model PO)
keep_main <- grepl("^(emp_status|tv_class|Age_Raw|education|industry|income|residence_len|gender)",
                   rownames(tab_all))
main_tab  <- tab_all[keep_main, , drop = FALSE]

# 4b) Koefisien 'jobrel' pada PPO adalah per-ambang (cutpoint-specific).
#     Kita ambil semua baris yang mengandung 'jobrel' → hitung OR per ambang.
keep_jobrel <- grepl("^jobrel", rownames(tab_all))
jobrel_tab  <- tab_all[keep_jobrel, , drop = FALSE]

# 5) Hitung OR, CI95, p
fmt_tbl <- function(tab) {
  beta <- tab[,1]; se <- tab[,2]; p <- tab[,4]
  data.frame(
    Term = rownames(tab),
    OR   = exp(beta),
    L95  = exp(beta - 1.96*se),
    U95  = exp(beta + 1.96*se),
    p    = p,
    row.names = NULL
  )
}

res_main   <- fmt_tbl(main_tab) %>%
  mutate(Comparison = case_when(
    Term == "emp_statusPart-time"              ~ "Part-time vs Full-time",
    Term == "emp_statusNot working"            ~ "Not working vs Full-time",
    Term == "tv_classPiloting"                 ~ "Piloting vs Advanced",
    Term == "tv_classDeveloping"               ~ "Developing vs Advanced",
    Term == "tv_classNot classified"           ~ "Not classified vs Advanced",
    Term == "Age_Raw"                          ~ "Age (per year)",
    Term == "educationUpperSec"                ~ "UpperSec vs Higher",
    Term == "educationLowerSec"                ~ "LowerSec vs Higher",
    Term == "educationNoPrimary"               ~ "No/Primary vs Higher",
    Term == "industryAgriculture/fisheries"    ~ "Agriculture/fisheries vs Other",
    Term == "industryManufacturing"            ~ "Manufacturing vs Other",
    Term == "industryPublic sector"            ~ "Public sector vs Other",
    Term == "industryServices"                 ~ "Services vs Other",
    Term == "industryTourism"                  ~ "Tourism vs Other",
    grepl("^income", Term)                     ~ sub("^income", "", Term),
    grepl("^residence_len", Term)              ~ sub("^residence_len", "", Term),
    grepl("^gender", Term)                     ~ sub("^gender", "", Term),
    TRUE ~ Term
  )) %>%
  dplyr::select(Comparison, OR, L95, U95, p)

# 6) Untuk jobrel (PPO): tampilkan OR per ambang (namanya biasanya mengandung label ambang).
#    Biarkan nama aslinya agar jelas ambang mana; nanti kita rangkum rentangnya.
res_jobrel <- fmt_tbl(jobrel_tab)

# 7) Cetak ringkas:
cat("\n== LRT PO vs PPO (jobrel only) ==\n")
print(anova(m_full, m_ppo), digits = 4)

cat("\n== Efek utama (selain jobrel) – OR cluster-robust ==\n")
print(
  res_main %>% filter(Comparison %in% c(
    "Part-time vs Full-time","Not working vs Full-time",
    "Piloting vs Advanced","Developing vs Advanced",
    "Age (per year)"
  )),
  digits = 4
)

cat("\n== jobrel (PPO) – OR per ambang (cluster-robust) ==\n")
print(res_jobrel, digits = 4)

# 8) (opsional) rangkum 'kisaran' OR untuk jobrel per kategori
rng <- res_jobrel %>%
  mutate(group = ifelse(grepl("Indirectly", Term), "Indirectly vs Unrelated",
                        ifelse(grepl("Directly", Term), "Directly vs Unrelated", "Other"))) %>%
  group_by(group) %>%
  summarise(OR_min = min(OR, na.rm=TRUE), OR_max = max(OR, na.rm=TRUE),
            L95_min = min(L95, na.rm=TRUE), U95_max = max(U95, na.rm=TRUE),
            .groups = "drop")

cat("\n== Ringkasan kisaran OR jobrel (per ambang) ==\n")
print(rng, digits = 4)


Warning message:
“(-1) Model failed to converge with max|grad| = 120.108 (tol = 1e-06) 
In addition: maximum number of consecutive Newton modifications reached”


,no.par,AIC,logLik,LR.stat,df,Pr(>Chisq)
,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
m_full,31,865.9607,-401.9803,NA,NA,NA
m_ppo,37,859.4423,-392.7212,18.51837,6,0.005059293


ERROR: Error in estfun.clm(x, ...): estimating functions for scale regression not implemented yet


In [31]:
# === OR per ambang untuk jobrel dari model PPO (SE Wald, non-cluster) ===
summ_ppo <- summary(m_ppo)
co_ppo   <- summ_ppo$coefficients

# Ambil baris koefisien yang terkait jobrel (threshold-specific)
job_idx  <- grepl("^jobrel", rownames(co_ppo))
job_tab  <- co_ppo[job_idx, , drop = FALSE]

beta <- job_tab[, "Estimate"]
se   <- job_tab[, "Std. Error"]
z    <- beta / se
p    <- 2 * pnorm(abs(z), lower.tail = FALSE)

res_jobrel_wald <- data.frame(
  Term = rownames(job_tab),
  OR   = exp(beta),
  L95  = exp(beta - 1.96 * se),
  U95  = exp(beta + 1.96 * se),
  p    = p,
  row.names = NULL
)

# Tampilkan tabel per-ambang (namanya memang menyertakan ambang cutpoint)
print(res_jobrel_wald, digits = 4)

# Ringkas kisaran OR per kelompok (Indirectly/Directly)
library(dplyr)
rng_jobrel <- res_jobrel_wald %>%
  mutate(group = case_when(
    grepl("Indirect", Term) ~ "Indirectly vs Unrelated",
    grepl("Direct",   Term) ~ "Directly vs Unrelated",
    TRUE                    ~ "Other"
  )) %>%
  group_by(group) %>%
  summarise(
    OR_min  = min(OR,  na.rm = TRUE),
    OR_max  = max(OR,  na.rm = TRUE),
    L95_min = min(L95, na.rm = TRUE),
    U95_max = max(U95, na.rm = TRUE),
    .groups = "drop"
  )

cat("\n== Ringkasan kisaran OR jobrel (per ambang, SE Wald) ==\n")
print(rng_jobrel, digits = 4)


                      Term OR L95 U95  p
1 jobrelIndirectly related NA  NA  NA NA
2   jobrelDirectly related NA  NA  NA NA


Warning message:
“There were 8 warnings in `summarise()`.
The first warning was:
ℹ In argument: `OR_min = min(OR, na.rm = TRUE)`.
ℹ In group 1: `group = "Directly vs Unrelated"`.
Caused by warning in `min()`:
! no non-missing arguments to min; returning Inf
ℹ Run `dplyr::last_dplyr_warnings()` to see the 7 remaining warnings.”



== Ringkasan kisaran OR jobrel (per ambang, SE Wald) ==
# A tibble: 2 × 5
  group                   OR_min OR_max L95_min U95_max
  <chr>                    <dbl>  <dbl>   <dbl>   <dbl>
1 Directly vs Unrelated      Inf   -Inf     Inf    -Inf
2 Indirectly vs Unrelated    Inf   -Inf     Inf    -Inf


In [32]:
library(dplyr)
library(ordinal)
library(sandwich)
library(lmtest)

# Subset strict insider: keep only Directly & Unrelated
dat_strict <- df6 %>%
  dplyr::select(
    overall_impact, emp_status, jobrel, tv_class,
    Age_Raw, education, industry,
    income, residence_len, gender,
    Weight, Village
  ) %>%
  tidyr::drop_na() %>%
  dplyr::filter(jobrel %in% c("Directly related","Unrelated")) %>%
  droplevels()

# Refit CLM (weighted)
m_strict <- clm(
  overall_impact ~ emp_status + jobrel + tv_class +
    Age_Raw + education + industry +
    income + residence_len + gender,
  data = dat_strict, weights = Weight, link = "logit"
)

# Cluster-robust SE by Village
Vcl_strict <- sandwich::vcovCL(m_strict, cluster = dat_strict$Village)
ct_strict  <- lmtest::coeftest(m_strict, vcov = Vcl_strict)

# Ambil satu efek kunci: Directly vs Unrelated
row <- "jobrelDirectly related"
beta <- ct_strict[row, 1]; se <- ct_strict[row, 2]; p <- ct_strict[row, 4]
OR   <- exp(beta); L95 <- exp(beta - 1.96*se); U95 <- exp(beta + 1.96*se)

data.frame(
  Comparison = "Directly vs Unrelated (strict insider)",
  OR = OR, L95 = L95, U95 = U95, p = p
)


Comparison,OR,L95,U95,p
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Directly vs Unrelated (strict insider),2.475861,1.235403,4.961851,0.01090569


In [33]:
library(dplyr)
library(ordinal)
library(sandwich)
library(lmtest)

# === 0) Mulai dari df6 yang sudah punya faktor siap pakai ===
# Proxy flags
dat_e2 <- df6 %>%
  dplyr::select(
    overall_impact, emp_status, jobrel, tv_class,
    Age_Raw, education, industry,
    income, residence_len, gender,
    Weight, Village
  ) %>%
  tidyr::drop_na() %>%
  mutate(
    student_proxy  = Age_Raw <= 22 & emp_status %in% c("Not working","Part-time"),
    retiree_proxy  = Age_Raw >  60
  )

# Ringkas jumlah kasus yang akan dikecualikan
n0   <- nrow(dat_e2)
n_st <- sum(dat_e2$student_proxy, na.rm = TRUE)
n_rt <- sum(dat_e2$retiree_proxy, na.rm = TRUE)

cat("N awal        :", n0, "\n")
cat("Student proxy :", n_st, "\n")
cat("Retiree proxy :", n_rt, "\n")

# Filter out students & retirees
dat_e2f <- dat_e2 %>%
  filter(!student_proxy, !retiree_proxy) %>%
  droplevels()

cat("N setelah eksklusi :", nrow(dat_e2f), "\n")

# === 1) Fit CLM berbobot (spesifikasi sama dengan model utama) ===
m_e2 <- clm(
  overall_impact ~ emp_status + jobrel + tv_class +
    Age_Raw + education + industry +
    income + residence_len + gender,
  data   = dat_e2f,
  weights = Weight,
  link   = "logit"
)

# === 2) Cluster-robust SE by Village ===
V_cl_e2 <- sandwich::vcovCL(m_e2, cluster = dat_e2f$Village)
ct_e2   <- lmtest::coeftest(m_e2, vcov = V_cl_e2)

# Ambil 4 efek utama
keep <- rownames(ct_e2) %in% c(
  "emp_statusPart-time",
  "emp_statusNot working",
  "jobrelIndirectly related",
  "jobrelDirectly related"
)
tab4 <- ct_e2[keep, , drop = FALSE]

beta <- tab4[,1]; se <- tab4[,2]; p <- tab4[,4]
OR   <- exp(beta); L95 <- exp(beta - 1.96*se); U95 <- exp(beta + 1.96*se)

res_e2 <- data.frame(
  Comparison = c("Part-time vs Full-time",
                 "Not working vs Full-time",
                 "Indirectly vs Unrelated",
                 "Directly vs Unrelated"),
  OR = OR, L95 = L95, U95 = U95, p = p,
  row.names = NULL
)

print(res_e2, digits = 4)


N awal        : 570 
Student proxy : 94 
Retiree proxy : 0 
N setelah eksklusi : 476 


Warning message:
“(1) Hessian is numerically singular: parameters are not uniquely determined 
In addition: Absolute convergence criterion was met, but relative criterion was not met”


                Comparison     OR L95 U95  p
1   Part-time vs Full-time 0.5233  NA  NA NA
2 Not working vs Full-time 0.4120  NA  NA NA
3  Indirectly vs Unrelated 2.0658  NA  NA NA
4    Directly vs Unrelated 2.6005  NA  NA NA


In [34]:
library(dplyr)
library(ordinal)
library(sandwich)
library(lmtest)

# Mulai dari dat_e2f (sudah exclude students/retirees) yang Anda buat barusan:
# (kalau belum ada di sesi, buat ulang ringkas)
dat_e2f <- df6 %>%
  dplyr::select(
    overall_impact, emp_status, jobrel, tv_class,
    Age_Raw, education, residence_len,  # <-- simpan inti; drop income/industry/gender agar stabil
    Weight, Village
  ) %>%
  tidyr::drop_na() %>%
  mutate(
    student_proxy  = Age_Raw <= 22 & emp_status %in% c("Not working","Part-time"),
    retiree_proxy  = Age_Raw > 60
  ) %>%
  filter(!student_proxy, !retiree_proxy) %>%
  droplevels()

# Fit CLM (logit) berbobot
m_e2_lean <- clm(
  overall_impact ~ emp_status + jobrel + tv_class +
    Age_Raw + education + residence_len,
  data   = dat_e2f,
  weights = Weight,
  link   = "logit"
)

# SE klaster per desa
Vcl_e2_lean <- sandwich::vcovCL(m_e2_lean, cluster = dat_e2f$Village)
ct_e2_lean  <- lmtest::coeftest(m_e2_lean, vcov = Vcl_e2_lean)

# Ambil 4 efek utama
keep <- rownames(ct_e2_lean) %in% c(
  "emp_statusPart-time",
  "emp_statusNot working",
  "jobrelIndirectly related",
  "jobrelDirectly related"
)
tab4 <- ct_e2_lean[keep, , drop = FALSE]

beta <- tab4[,1]; se <- tab4[,2]; p <- tab4[,4]
OR   <- exp(beta); L95 <- exp(beta - 1.96*se); U95 <- exp(beta + 1.96*se)

res_e2_lean <- data.frame(
  Comparison = c("Part-time vs Full-time",
                 "Not working vs Full-time",
                 "Indirectly vs Unrelated",
                 "Directly vs Unrelated"),
  OR = OR, L95 = L95, U95 = U95, p = p,
  row.names = NULL
)

print(res_e2_lean, digits = 4)


                Comparison     OR    L95    U95         p
1   Part-time vs Full-time 0.5687 0.2925 1.1057 0.0968287
2 Not working vs Full-time 0.5273 0.3020 0.9204 0.0247961
3  Indirectly vs Unrelated 2.3825 1.3956 4.0673 0.0015611
4    Directly vs Unrelated 2.6043 1.4996 4.5226 0.0007332


In [35]:
library(dplyr)
library(sandwich)
library(lmtest)

set.seed(123)

# 1) Siapkan data cluster dari 4 dimensi dampak
dat_km <- df6 %>%
  dplyr::select(
    overall_impact,           # untuk labeling sanity-check
    Econ_Change, Soc_Change, Env_Change, Gov_Change,
    emp_status, jobrel, tv_class,
    Age_Raw, education, residence_len,
    Weight, Village
  ) %>%
  tidyr::drop_na()

# Skala variabel dampak (mean=0, sd=1)
X <- scale(dat_km[, c("Econ_Change","Soc_Change","Env_Change","Gov_Change")])

# 2) K-means 2 cluster
km <- kmeans(X, centers = 2, nstart = 50)

# 3) Label cluster: "optimist" = cluster dengan rata-rata overall_impact tertinggi
dat_km$cluster <- factor(km$cluster)
lab <- dat_km %>%
  group_by(cluster) %>%
  summarise(mean_overall = mean(as.numeric(overall_impact), na.rm=TRUE), .groups="drop") %>%
  arrange(desc(mean_overall)) %>%
  mutate(label = c("optimist","critic"))

map_lab <- setNames(lab$label, lab$cluster)
dat_km$cluster_lab <- factor(map_lab[dat_km$cluster], levels = c("critic","optimist"))  # ref=critic

# 4) Ringkas cross-tab cepat (cek pola kasar)
cat("\n== Crosstab jobrel x cluster ==\n")
print(dat_km %>% count(jobrel, cluster_lab) %>% tidyr::pivot_wider(names_from = cluster_lab, values_from = n, values_fill = 0))

cat("\n== Crosstab emp_status x cluster ==\n")
print(dat_km %>% count(emp_status, cluster_lab) %>% tidyr::pivot_wider(names_from = cluster_lab, values_from = n, values_fill = 0))

# 5) Logit berbobot: P(optimist) ~ jobrel + emp_status + kovariat inti ; SE klaster per Village
#    (pakai family=binomial, weights=Weight, vcovCL untuk cluster by Village)
dat_km$y_opt <- as.integer(dat_km$cluster_lab == "optimist")

m_km <- glm(
  y_opt ~ jobrel + emp_status + tv_class + Age_Raw + education + residence_len,
  family = binomial(link = "logit"),
  data   = dat_km,
  weights = Weight
)

V_km <- sandwich::vcovCL(m_km, cluster = dat_km$Village)
ct_km <- lmtest::coeftest(m_km, vcov = V_km)

# Ambil 4 efek utama yang kita pedulikan
rows <- c("emp_statusPart-time","emp_statusNot working","jobrelIndirectly related","jobrelDirectly related")
tab4 <- ct_km[rows, , drop = FALSE]

beta <- tab4[,1]; se <- tab4[,2]; p <- tab4[,4]
OR   <- exp(beta); L95 <- exp(beta - 1.96*se); U95 <- exp(beta + 1.96*se)

res_km <- data.frame(
  Comparison = c("Part-time vs Full-time (optimist odds)",
                 "Not working vs Full-time (optimist odds)",
                 "Indirectly vs Unrelated (optimist odds)",
                 "Directly vs Unrelated (optimist odds)"),
  OR = OR, L95 = L95, U95 = U95, p = p,
  row.names = NULL
)

cat("\n== Logit hasil utama (cluster-robust by Village) ==\n")
print(res_km, digits = 4)



== Crosstab jobrel x cluster ==
# A tibble: 3 × 3
  jobrel             critic optimist
  <fct>               <int>    <int>
1 Unrelated              29      342
2 Indirectly related      8       63
3 Directly related       21      107

== Crosstab emp_status x cluster ==
# A tibble: 3 × 3
  emp_status  critic optimist
  <fct>        <int>    <int>
1 Full-time       19      158
2 Part-time       30      194
3 Not working      9      160


Warning message in eval(family$initialize):
“non-integer #successes in a binomial glm!”



== Logit hasil utama (cluster-robust by Village) ==
                                Comparison     OR    L95    U95       p
1   Part-time vs Full-time (optimist odds) 0.7324 0.4182 1.2827 0.27605
2 Not working vs Full-time (optimist odds) 2.3375 0.8018 6.8144 0.11984
3  Indirectly vs Unrelated (optimist odds) 0.7997 0.2957 2.1623 0.65956
4    Directly vs Unrelated (optimist odds) 0.3280 0.1319 0.8157 0.01647
